In [1]:
from pynterp.rules.decision_set import DecisionSet 
import numpy as np
import pandas as pd
import Orange
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

Because the datasets in the decision paper are not public, we instead choose to compare to the CN2 baseline (see [here for more details](https://orange.readthedocs.io/en/latest/reference/rst/Orange.classification.rules.html)).

In [2]:
# Read some data
titanic = Orange.data.Table("titanic")

In [3]:
titanic

[[first, adult, male | yes],
 [first, adult, male | yes],
 [first, adult, male | yes],
 [first, adult, male | yes],
 [first, adult, male | yes],
 ...
]

In [4]:
titanic.X

array([[1., 0., 1.],
       [1., 0., 1.],
       [1., 0., 1.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

The CN2 rule-based classifier (provided by Orange)

In [5]:
# construct the learning algorithm and use it to induce a classifier
cn2_learner = Orange.classification.rules.CN2Learner()
cn2_clasifier = cn2_learner(titanic)
for r in cn2_clasifier.rule_list:
    print(r)

IF sex==female AND status==first AND age!=adult THEN survived=yes 
IF sex==female AND status!=third AND age!=adult THEN survived=yes 
IF sex!=female AND status==second AND age!=adult THEN survived=yes 
IF sex==female AND status==first THEN survived=yes 
IF status!=third AND age!=adult THEN survived=yes 
IF sex!=female AND status==second THEN survived=no 
IF status==crew AND sex!=male THEN survived=yes 
IF status==second THEN survived=yes 
IF sex!=female AND status==third AND age!=child THEN survived=no 
IF status==crew THEN survived=no 
IF sex!=female AND status!=first THEN survived=no 
IF status==first THEN survived=no 
IF age!=adult THEN survived=no 
IF status==third THEN survived=no 
IF TRUE THEN survived=no 


In [6]:
def table_to_pandas(titanic):
    X, y = titanic.X, titanic.Y
    X = pd.DataFrame(X, columns=['status', 'age', 'sex'])
    y = pd.Series(y)
    for s in X['status'].unique():
        X['status_%d' % s] = X['status'].apply(lambda x: 1 if x == s else 0)
    X.drop('status', axis=1, inplace=True)
    return X, y

In [7]:
X, y = table_to_pandas(titanic)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.05, random_state=43)
print('Data shapes: ', X_train.shape, X_val.shape, X_test.shape)

In [11]:
cn2_preds = np.argmax(cn2_clasifier.predict(X_test.values), axis=1)

In [12]:
cn2_acc = accuracy_score(y_test, cn2_preds)
print('CN2 Accuracy: {}'.format(cn2_acc))

CN2 Accuracy: 0.272108843537415


In [44]:
lr = LogisticRegressionCV().fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_acc = accuracy_score(y_test, lr_pred)
print('Logistic Regression Accuracy: {}'.format(lr_acc))

Logistic Regression Accuracy: 0.7732426303854876


/Users/dylanrandle/anaconda3/envs/irl/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


CN2 has very bad accuracy compared to logistic regression.

# Heuristic Choice of Hyperparamaters ($\lambda_i$)

Fitting using heuristically chosen hyperparameters (100 for any objective elements without a large constant addition (e.g. f5 and f7), 1 for those with a large constant addition (e.g. f1,f2,f3...)

In [13]:
ds = DecisionSet(min_support=0.01, lambdas=[1, 1, 1, 1, 100, 1, 100])

In [14]:
ds.fit(X_train, y_train)

Rules in domain = 28
Preprocessing...
Done
Error bound = 1430.5752551020407
Estimating w...
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (15, 2)
Current A:
               item  class
2             (sex)    1.0
3             (sex)    0.0
4        (status_1)    1.0
6        (status_2)    1.0
8        (status_3)    1.0
9        (status_3)    0.0
10       (status_0)    1.0
11       (status_0)    0.0
18  (status_1, sex)    1.0
19  (status_1, sex)    0.0
21  (status_2, sex)    0.0
22  (status_3, sex)    1.0
23  (status_3, sex)    0.0
24  (status_0, sex)    1.0
25  (status_0, sex)    0.0
Estimating w...
Done
Adding elements to A 
Executed _add_elements, shape change of A: (15, 2) -> (15, 2)
Current A:
               item  class
2             (sex)    1.0
3             (sex)    0.0
4        (status_1)    1.0
6        (status_2)    1.0
8        (status_3)    1.0
9        (status_3)    0.0
10       (status_0)    1.0
11       (status_0)    0.0
18  (status_1, sex

In [15]:
ds.decision_set

,item,class,acc
14,"(age, status_2)",1.0,1.000000
23,"(status_3, sex)",0.0,0.831266
3,(sex),0.0,0.789553
25,"(status_0, sex)",0.0,0.788253
9,(status_3),0.0,0.750449
27,"(age, status_3, sex)",0.0,0.711111
4,(status_1),1.0,0.616667
7,(status_2),0.0,0.566667
20,"(status_2, sex)",1.0,0.167939


In [16]:
# age == 0 => adult
# age == 1 => child
# sex == 0 => female
# sex == 1 => male
# class == 0 => crew ??
# class == 1 => first
# class == 2 => second
# class == 3 => third

def item_to_str(item):
    if item=='age':
        out = 'child'
    elif item=='sex':
        out = 'male'
    elif item=='status_0':
        out = 'crew'
    elif item=='status_1':
        out = 'first_class'
    elif item=='status_2':
        out = 'second_class'
    else:
        out = 'third_class'
    return out

def rule_as_str(rule):
    itemset = rule['item']
    out='IF'
    for i, item in enumerate(itemset):
        if i == 0:
            out += ' {}'.format(item_to_str(item))
        else:
            out += ' AND {}'.format(item_to_str(item))
    outcome = 'Survive' if rule['class']==1 else 'Perish'
    return out + ' THEN ' + outcome

def print_decision_set(dset, else_pred='Perish'):
    for i in range(len(dset)):
        r = dset.iloc[i]
        print(rule_as_str(r))
    print('ELSE {}'.format(else_pred))

In [18]:
print_decision_set(ds.decision_set)

IF child AND second_class THEN Survive
IF third_class AND male THEN Perish
IF male THEN Perish
IF crew AND male THEN Perish
IF third_class THEN Perish
IF child AND third_class AND male THEN Perish
IF first_class THEN Survive
IF second_class THEN Perish
IF second_class AND male THEN Survive
ELSE Perish


In [19]:
ds_pred = ds.predict(X_test)
ds_acc = accuracy_score(y_test, ds_pred)
print('Decision Set Accuracy: {}'.format(ds_acc))

Decision Set Accuracy: 0.7346938775510204


Our decision set achieves comparable accuracy to logistic regression, and much higher than CN2.

# Automated Hyperparameter Selection:

In [45]:
from copy import deepcopy
from multiprocessing import Pool, Process

def hyperparam_search(lambda_bounds=[(0,100), (0, 100), (0, 100), (0, 100), (0, 100), (0, 100), (0, 100)], 
                      min_support=0.01):
    """
    perform search of lambda space based on a ternary search strategy
    """
    done = False
    best_acc = -1e6
    best_ds = None
    best_hypers = None
    lambdas = [1, 1, 1, 1, 1, 1, 1]
    while not done:
        for i in range(len(lambda_bounds)):
            # update one at a time
            l, r = lambda_bounds[i][0], lambda_bounds[i][1]
            mid1 = l + (r-l)/3
            mid2 = r - (r-l)/3
            lambdas[i] = mid1
            dset1 = DecisionSet(min_support=min_support, lambdas=lambdas).fit(X_train, y_train)
            acc1 = accuracy_score(y_val, dset1.predict(X_val))
            lambdas[i] = mid2
            dset2 = DecisionSet(min_support=min_support, lambdas=lambdas).fit(X_train, y_train)
            acc2 = accuracy_score(y_val, dset2.predict(X_val))
            last_best = best_acc
            if acc2 > acc1:
                lambda_bounds[i] = (mid1, r)
                if acc2 > best_acc:
                    best_acc = acc2
                    best_ds = deepcopy(dset2)
                    best_hypers = dset2.lambdas
            else:
                lambda_bounds[i] = (l, mid2)
                if acc1 > best_acc:
                    best_acc = acc1
                    best_ds = deepcopy(dset1)
                    best_hypers = dset1.lambdas
            print('New best: {}, {}'.format(best_acc, best_hypers))
            if abs(last_best-best_acc)/best_acc < 0.01:
                done = True
    
    print('Final best: {}, {}'.format(best_acc, best_hypers))
    return best_ds, best_hypers, best_acc

def search_over_params(param_list, niters=100, lambda_min=0, lambda_max=100, 
                  min_support=0.01, score=accuracy_score):
    """
    search over param_list for best params
    """
    best_score = -1e6
    best_lambda = None
    best_ds = None
        
    def fit_ds(lambdas):
        ds = DecisionSet(min_support=min_support, lambdas=lambdas)
        ds.fit(X_train, y_train)
        ds_val = ds.predict(X_val)
        s = score(y_val, ds_val)
        return deepcopy(ds), lambdas, s
        
    for l in param_list:
        r = fit_ds(l)
        print(r)
        model, hypers, _score = r
        if _score > best_score:
            print('New best: {}'.format(r))
            best_score = _score
            best_lambda = hypers
            best_ds = model
                
    print('Final best: {}, score={}'.format(best_lambda, best_score))
    return best_ds, best_lambda, best_score

## Parallelized Random Search

In [23]:
n_cpus = 8
iter_per_cpu = 100
random_search_space = [[np.random.randint(0,1000,size=7) for _ in range(iter_per_cpu)] for _ in range(n_cpus)]

In [24]:
with Pool(n_cpus) as p:
    result = p.map(search_over_params, random_search_space)

Rules in domain = 28
Rules in domain = 28
Rules in domain = 28
Rules in domain = 28
Rules in domain = 28
Rules in domain = 28
Rules in domain = 28
Preprocessing...
Preprocessing...
Preprocessing...
Preprocessing...
Rules in domain = 28
Preprocessing...
Preprocessing...
Preprocessing...
Preprocessing...
Done
Done
Done
Done
Done
Done
Done
Done
Error bound = 577190.762755102
Estimating w...
Error bound = 269803.66709183675
Estimating w...
Error bound = 866016.2576530612
Estimating w...
Error bound = 373983.0293367347
Estimating w...
Error bound = 501745.4375
Error bound = 685913.7704081633
Estimating w...
Estimating w...
Error bound = 889315.4591836735
Estimating w...
Error bound = 689451.6632653062
Estimating w...
Done
Done
Adding elements to A 
Adding elements to A 
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (0, 2)
Executed _add_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []Done
Current A:
Em

Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Calculating individual rule scores
Done
Done
Done
Error bound = 555814.1951530612
Error bound = 567738.7831632653
Estimating w...
Estimating w...
Done
Error bound = 717042.6288265307
Estimating w...
Error bound = 563331.1721938775
Estimating w...
(<irl.decision_set.DecisionSet object at 0x120e96320>, array([595, 174, 691, 200, 928, 795, 928]), 0.5454545454545454)
Rules in domain = 28
Preprocessing...
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Removing elements from A
Executed _remove_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Calculating individual rule scores
(<irl.decision_set.DecisionSet object at 0x12079b908>, array([658, 351, 778, 156, 357, 745, 989]), 0.8068181818181818)
New best: (<irl.decision_set.DecisionSet object at 0x12079b908>, array([658, 351, 778, 

Preprocessing...
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Removing elements from A
Executed _remove_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Calculating individual rule scores
Done
Done
Error bound = 776905.381377551
Estimating w...
Error bound = 816203.0306122449
Estimating w...
(<irl.decision_set.DecisionSet object at 0x1206952b0>, array([537, 770, 903, 727, 587,  82, 632]), 0.7613636363636364)
Rules in domain = 28
Preprocessing...
Done
Error bound = 524072.4349489796
Estimating w...
Done
Error bound = 833773.5395408163
Estimating w...
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Removing elements from A
Executed _remove_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]


Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Calculating individual rule scores
(<irl.decision_set.DecisionSet object at 0x120e72a90>, array([573, 570,  42,  54, 266, 445, 846]), 0.7613636363636364)
Rules in domain = 28
Preprocessing...
(<irl.decision_set.DecisionSet object at 0x120a04eb8>, array([587, 199,  93, 377, 114, 806, 915]), 0.7386363636363636)
Rules in domain = 28
Preprocessing...
Done
Error bound = 856898.1198979592
Estimating w...
Done
Error bound = 842973.6428571428
Estimating w...
Done
Error bound = 519997.7461734694
Estimating w...
Done
Error bound = 341570.2653061224
Estimating w...
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Removing elements from A
Executed _remove_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Calculating individual rule scores
(<irl.decision_set.DecisionSet object at 0x1212cf

Index: []
Calculating individual rule scores
Calculating individual rule scores
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Removing elements from A
Executed _remove_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Removing elements from A
Calculating individual rule scores
Executed _remove_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
(<irl.decision_set.DecisionSet object at 0x120e69208>, array([633, 295, 634, 681, 413, 218, 771]), 0.6704545454545454)
Rules in domain = 28
Preprocessing...
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
I

...

Estimating w...
Calculating individual rule scores
Done
Calculating individual rule scores
Error bound = 807982.0102040817
Estimating w...
Done
Error bound = 493877.3405612245
Estimating w...
(<irl.decision_set.DecisionSet object at 0x120697710>, array([106,  95, 573, 987, 912,   9, 953]), 0.8068181818181818)
Rules in domain = 28
Preprocessing...
Done
(<irl.decision_set.DecisionSet object at 0x1212cf5c0>, array([965, 838, 695, 106,  48,  59, 123]), 0.25)
Rules in domain = 28
Preprocessing...
Error bound = 777730.1658163265
Estimating w...
Done
Error bound = 952797.8966836735
Estimating w...
Done
Error bound = 755819.0650510204
Estimating w...
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []Done

Removing elements from A
Executed _remove_elements, shape change of A: (0, 2) -> (0, 2)
Error bound = 403505.51020408166
Current A:
Empty DataFrame
Columns: [item, class]
Index: []Estimating w...



Preprocessing...
Calculating individual rule scores
Calculating individual rule scores
Done
Error bound = 549528.3954081633
Estimating w...
(<irl.decision_set.DecisionSet object at 0x1212c4f98>, array([596, 658, 246, 652,  92, 726,  56]), 0.4659090909090909)
Rules in domain = 28
Preprocessing...
(<irl.decision_set.DecisionSet object at 0x11f942e10>, array([ 66, 345, 976, 380, 812,  81, 617]), 0.4431818181818182)
Rules in domain = 28
Preprocessing...
Done
Error bound = 802401.112244898
Estimating w...
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Removing elements from A
Executed _remove_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Calculating individual rule scores
Done
Error bound = 488342.6275510204
Estimating w...
Done
Error bound = 572697.5765306122
Estimating w...
(<irl.decision_set.DecisionSet object at 0x1206aa0f0>, arr

Index: []
Removing elements from A
Executed _remove_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Calculating individual rule scores
Done
Error bound = 673675.0459183673
Estimating w...
(<irl.decision_set.DecisionSet object at 0x12079bb70>, array([243, 706, 345, 229,  72, 897, 897]), 0.3409090909090909)
Rules in domain = 28
Preprocessing...
Done
Error bound = 220466.9043367347
Estimating w...
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Removing elements from A
Executed _remove_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Calculating individual rule scores
Done
Error bound = 510219.69770408166
Estimating w...
(<irl.decision_set.DecisionSet object at 0x120e6bc88>, array([829, 202, 764, 985, 109, 732, 796]), 0.75)
Rules in domain = 28
Preprocessing...
Done
Adding eleme

Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Calculating individual rule scores
Done
Error bound = 636205.9234693878
Estimating w...
(<irl.decision_set.DecisionSet object at 0x120a00550>, array([560, 548, 781, 225, 124, 263, 253]), 0.6477272727272727)
Rules in domain = 28
Preprocessing...
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Removing elements from A
Executed _remove_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Calculating individual rule scores
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Removing elements from A
Executed _remove_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Done
Error bound = 905095.3954081633
Estimating w...
Calculating indiv

Rules in domain = 28
Preprocessing...
Done
Error bound = 274208.0191326531
Estimating w...
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Removing elements from A
Executed _remove_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Calculating individual rule scores
(<irl.decision_set.DecisionSet object at 0x120a82828>, array([511, 728, 583, 474, 564, 952, 806]), 0.7954545454545454)
Rules in domain = 28
Preprocessing...
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Removing elements from A
Executed _remove_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Done
Error bound = 794788.1275510204
Estimating w...
Calculating individual rule scores
(<irl.decision_set.DecisionSet object a

# Overall best decision set (from random search)

In [29]:
ds_ransearch = result[np.argmax([ele[2] for ele in result])][0]

In [35]:
ds_ransearch.lambdas

array([186, 732, 160, 950, 800, 752, 281])

In [30]:
ds_ransearch.decision_set

,item,class,acc
21,"(status_2, sex)",0.0,0.832061
23,"(sex, status_3)",0.0,0.831266
3,(sex),0.0,0.789553
19,"(sex, status_1)",0.0,0.635714
17,"(age, status_3)",0.0,0.633803
4,(status_1),1.0,0.616667
12,"(age, sex)",1.0,0.448276
6,(status_2),1.0,0.433333
26,"(sex, age, status_3)",1.0,0.288889
10,(status_0),1.0,0.230075


In [34]:
accuracy_score(y_test, ds_ransearch.predict(X_test))

0.7755102040816326

Random search leads to model with higher accuracy than logistic!

In [33]:
print_decision_set(ds_ransearch.decision_set)

IF second_class AND male THEN Perish
IF male AND third_class THEN Perish
IF male THEN Perish
IF male AND first_class THEN Perish
IF child AND third_class THEN Perish
IF first_class THEN Survive
IF child AND male THEN Survive
IF second_class THEN Survive
IF male AND child AND third_class THEN Survive
IF crew THEN Survive
IF child AND second_class THEN Perish
ELSE Perish


## Ternary Search

In [46]:
# this algorithm is not fully implemented due to time constraints, but still achieves fairly good results
result = hyperparam_search() 

Rules in domain = 28
Preprocessing...
Done
Error bound = 1240.862670068027
Estimating w...
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Removing elements from A
Executed _remove_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Calculating individual rule scores
Rules in domain = 28
Preprocessing...
Done
Error bound = 1238.0663265306123
Estimating w...
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Removing elements from A
Executed _remove_elements, shape change of A: (0, 2) -> (0, 2)
Current A:
Empty DataFrame
Columns: [item, class]
Index: []
Calculating individual rule scores
New best: 0.7613636363636364, [66.66666666666666, 1, 1, 1, 1, 1, 1]
Rules in domain = 28
Preprocessing...
Done
Error bound = 1242.7312925170068
Estimating w.

In [49]:
ds_tern = result[0]

In [50]:
accuracy_score(y_test, ds_tern.predict(X_test))

0.7687074829931972

In [51]:
print_decision_set(ds_tern.decision_set)

IF child AND second_class THEN Survive
IF third_class AND male THEN Perish
IF male THEN Perish
IF crew THEN Perish
IF first_class AND male THEN Perish
IF first_class THEN Survive
IF child THEN Survive
IF child AND male THEN Survive
IF second_class THEN Survive
IF crew AND male THEN Survive
IF second_class AND male THEN Survive
ELSE Perish


# Another heuristic method:

In [36]:
lambdas = [1/14, 3/14, 1/(1672*(14**2)), 1/(1672*(14**2)), 1, 1/(1672*14), 1]

In [37]:
ds = DecisionSet(min_support=0.01, lambdas=lambdas).fit(X_train, y_train)

Rules in domain = 28
Preprocessing...
Done
Error bound = 1.6582164323228559
Estimating w...
Done
Adding elements to A 
Executed _add_elements, shape change of A: (0, 2) -> (20, 2)
Current A:
               item  class
0             (age)    1.0
1             (age)    0.0
2             (sex)    1.0
3             (sex)    0.0
4        (status_1)    1.0
6        (status_2)    1.0
7        (status_2)    0.0
8        (status_3)    1.0
9        (status_3)    0.0
10       (status_0)    1.0
11       (status_0)    0.0
14  (age, status_2)    1.0
16  (age, status_3)    1.0
18  (status_1, sex)    1.0
19  (status_1, sex)    0.0
20  (status_2, sex)    1.0
21  (status_2, sex)    0.0
23  (status_3, sex)    0.0
24  (status_0, sex)    1.0
25  (status_0, sex)    0.0
Estimating w...
Done
Adding elements to A 
Executed _add_elements, shape change of A: (20, 2) -> (21, 2)
Current A:
               item  class
0             (age)    1.0
1             (age)    0.0
2             (sex)    1.0
3             (sex

In [38]:
ds.decision_set

,item,class,acc
14,"(age, status_2)",1.0,1.000000
25,"(status_0, sex)",0.0,0.788253
11,(status_0),0.0,0.769925
19,"(status_1, sex)",0.0,0.635714
17,"(age, status_3)",0.0,0.633803
4,(status_1),1.0,0.616667
7,(status_2),0.0,0.566667
0,(age),1.0,0.531250
8,(status_3),1.0,0.249551
2,(sex),1.0,0.210447


In [39]:
dspred = ds.predict(X_test)
accuracy_score(y_test, dspred)

0.6326530612244898

Worse than previous methods.

In [40]:
print_decision_set(ds.decision_set)

IF child AND second_class THEN Survive
IF crew AND male THEN Perish
IF crew THEN Perish
IF first_class AND male THEN Perish
IF child AND third_class THEN Perish
IF first_class THEN Survive
IF second_class THEN Perish
IF child THEN Survive
IF third_class THEN Survive
IF male THEN Survive
IF third_class AND male THEN Survive
IF second_class AND male THEN Survive
ELSE Perish
